# Topological Model of TJL and NC Twins

### Required Packages

In [1]:
import numpy as np
from numpy.linalg import inv,norm
from nmgtwinning import *
#import sympy as sp
#sp.init_printing()

### Metric Tensor & Structure Matrix

In [2]:
## Metric tensor, G
G = [[am**2, am*bm*np.cos(gamma),0],[am*bm*np.cos(gamma),bm**2,0],[0,0,cm**2]]

# V₁ lattice vectors in reference frame
ar = [am,0,0]
br = [bm*np.cos(gamma),bm*np.sin(gamma),0]
cr = [0,0,cm]

## ᵣPₘ
rPm = np.column_stack((ar,br,cr))
del ar,br,cr

### Burgers vector of compound twins

$V_1 \to V_3: (010)_{TB} = (\mathbf{b}_{13},h_{13})$

$V_3 \to V_4: (\bar{1}\bar{1}0)_{TB} = (\mathbf{b}_{34},h_{34})$

$V_1 \to V_2: (110)_{TB} = (\mathbf{b}_{12},h_{12})$

In [3]:
## obtain burgers vector and step height

#V₁ → V₃: (010) TB
tmu = rPm @ [0,1/2,1/2]
R = rotationmatrix([0,1,0] @ inv(rPm),np.pi)
tlamda = R @ rPm @ [0,1/2,-1/2]
b13 = norm(tlamda - tmu)
h13 = 1/np.sqrt([0,2,0] @ inv(G) @ [0,2,0])
del tmu,R,tlamda

#V₃ → V₄: (-1-10) TB
tmu = rPm @ [-1/2,0,-1/2]
R = rotationmatrix([-1,-1,0] @ inv(rPm),np.pi)
tlamda = R @ rPm @ [0,-1/2,1/2]
b34 = norm(tlamda - tmu)
h34 = 1/np.sqrt([-2,-2,0] @ inv(G) @ [-2,-2,0])
del tmu,R,tlamda

#V₁ → V₂: (110) TB
tmu = rPm @ [1/2,0,1/2]
R = rotationmatrix([1,1,0] @ inv(rPm),np.pi)
tlamda = R @ rPm @ [0,1/2,-1/2]
b12 = norm(tlamda - tmu)
h12 = 1/np.sqrt([2,2,0] @ inv(G) @ [2,2,0])
del tmu,R,tlamda

#print disconnections properties
print('b₁₃ = %6.4f \th₁₃ = %6.4f \nb₃₄ = %6.4f \th₃₄ = %6.4f \nb₁₂ = %6.4f \th₁₂ = %6.4f\n' % (b13,h13,b34,h34,b12,h12)) 

## minimization constant
k = (b13*h34)/(b34*h13)
print('k = (b₁₃ h₃₄) / (b₃₄ h₁₃) = %6.3f' % k)

b₁₃ = 0.0038 	h₁₃ = 0.2972 
b₃₄ = 0.0020 	h₃₄ = 0.2100 
b₁₂ = 0.0020 	h₁₂ = 0.2100

k = (b₁₃ h₃₄) / (b₃₄ h₁₃) =  1.374


### Angle between $(\bar{1} \bar{1} 0)$ and $(0 \bar{1} 0)$ plane

In [4]:
# Angle between (-1,-1,0) and (0,-1,0) plane
adotb = [-1,-1,0] @ inv(G) @ [0,-1,0]
a = np.sqrt([-1,-1,0] @ inv(G) @ [-1,-1,0])
b = np.sqrt([0,-1,0] @ inv(G) @ [0,-1,0])
psi = np.arccos(adotb/(a*b))
del a,b,adotb
print('ψ = %4.3f°' % np.degrees(psi)) 

ψ = 44.681°


### Equilibrium Tilt

#### Strain Equilibrium

We obtain the equilibrium misorientation $\theta$ at which the strain along the interface vanishes when $(\mathbf{b}_{13},h_{13})$ and $(\mathbf{b}_{34},h_{34})$ disconnections meet along a interface.

In [5]:
## Numerically calculate theta
## Minimization equation: sin 2(ψ − θ) - k sin 2θ = 0

## range of theta from 0 to π/2
theta = np.linspace(0, np.pi/2, 10000)

## obtain the equation value
eqn = []
for i in theta:
    dummy = np.abs(np.sin(2*(psi-i)) - k*np.sin(2*i))
    eqn.append(dummy)
    del dummy

## find the equation minimum
min_index = np.where(eqn == np.min(eqn)) # index with closest 0 value
theta = theta[min_index[0][0]] # theta for zero solution
print('θ = %4.3f°' % np.degrees(theta)) # print theta
del eqn, min_index

θ = 17.912°


#### Misorientation tilt

In [6]:
phi = b13*np.sin(theta)**2/h13
print('ϕ = %4.3f°' % np.degrees(phi))
phi_p = b34*np.sin(psi - theta)**2/h13
print('ϕ\' = %4.3f°' % np.degrees(phi_p))
Delta = phi_p + phi
print ('ϕ(avg) = %4.3f°' % (np.degrees(Delta)/2))
print('Δ = ϕ + ϕ\' = %4.4f°' % np.degrees(Delta))
del phi,phi_p

ϕ = 0.070°
ϕ' = 0.077°
ϕ(avg) = 0.074°
Δ = ϕ + ϕ' = 0.1472°


#### Final interface orientation

We obtain the final interface orientation with respect to $(010)

In [7]:
theta_TM = theta + Delta/2
print('θ(TM) = θ + Δ/2 = %4.3f°' % np.degrees(theta_TM)) 

θ(TM) = θ + Δ/2 = 17.985°


#### Other parameters

In [8]:
L = h13 / np.sin(theta)
L_p = h34 / np.sin(psi - theta) 
print('L = %4.4f nm' % L)
print('L\' = %4.4f nm' % L_p)

L = 0.9663 nm
L' = 0.4662 nm


### Classical Model Prediction

From classical model, the angle between $(010)_m$ and $(\bar{1} \bar{q} 0)_m$ gives us the equilibrium condition

In [9]:
## Irrational elements NC twins
q = (2*am*bm*np.cos(gamma) - np.sqrt(am**4 + bm**4 + 2*am**2 * bm**2 * np.cos(2*gamma)))/(am**2 - bm**2)
r = (2*am*bm*np.cos(gamma) + np.sqrt(am**4 + bm**4 + 2*am**2 * bm**2 * np.cos(2*gamma)))/(am**2 - bm**2)

## Anlge between (-1,-q,0) and (0,1,0) plane
adotb = [-1,-q,0] @ inv(G) @ [0,1,0]
a = np.sqrt([-1,-q,0] @ inv(G) @ [-1,-q,0])
b = np.sqrt([0,1,0] @ inv(G) @ [0,1,0])
theta_CM = np.arccos(adotb/(a*b))
print('θ(CM) = %4.3f°' % np.degrees(theta_CM))
del adotb,a,b

θ(CM) = 17.978°


In [10]:
print ('θ(TM) − θ(CM)= %4.4f°' % (np.degrees(theta_TM) - np.degrees(theta_CM)))

θ(TM) − θ(CM)= 0.0069°


### Angle between the TBs in QJL

In [11]:
## θ₁
theta1 = angleTBpair(rPm, [1, 1, 0], [1, r, 0])
print('θ₁ = %4.2f°' % round(np.degrees(theta1),5))

## θ₂
theta2 = angleTBpair(rPm, [1, q, 0], [-1, -1, 0])
print('θ₂ = %4.2f°' % round(np.degrees(theta2),5))

## θ₃
theta3 = angleTBpair(rPm, [-1, -1, 0], [1, r, 0])
print('θ₃ = %4.2f°' % round(np.degrees(theta3),5))

## θ₄
theta4 = angleTBpair(rPm, [1, q, 0], [1,1,0])
print('θ₄ = %4.2f°' % round(np.degrees(theta4),5))

## sum of angles
print('θ₁ + θ₂ + θ₃ + θ₄ = %4.2f°' % round(np.degrees(theta1 + theta2 + theta3 + theta4),5))
del theta1, theta2, theta3, theta4

θ₁ = 26.88°
θ₂ = 62.66°
θ₃ = 153.12°
θ₄ = 117.34°
θ₁ + θ₂ + θ₃ + θ₄ = 360.00°
